In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Cargar Datos

In [47]:
# Cargar los datos de entrenamiento
df_train = pd.read_csv('data/train_data.csv', sep=';')
df_train.head()

,id_colaborador,id_ultimo_jefe,seniority,modalidad_trabajo,distancia_oficina,dias_baja_salud,genero,canal_reclutamiento,permanencia_promedio,fecha_nacimiento,salario,performance_score,psi_score,fecha_incorporacion,estado_civil,abandono_6meses
0,100247,102074.0,1,Híbrida,1.760,1,Mujer,Ferias & Networking,1,25/11/1999,140011,99.0,75,25/01/2018,Soltero,0
1,103355,102115.0,1,Híbrida,0.760,2,Hombre,Ferias & Networking,2,16/10/1996,182774,99.0,81,24/07/2021,Soltero,1
2,100669,102060.0,1,Híbrida,4.950,3,Mujer,Referidos,11,29/01/1971,682106,96.0,74,13/07/2016,Viudo,0
3,103760,102062.0,1,Híbrida,13.030,2,Hombre,Linkedin,2,28/04/1992,270232,96.0,82,21/07/2014,Soltero,1
4,100965,102062.0,1,Híbrida,13.045,2,Hombre,Linkedin,2,3/05/1992,266804,95.0,82,22/07/2014,Soltero,0


## Preprocesamiento de datos

In [17]:
from utils import apply_base_preprocessing

# Hacer una limpieza basica de los datos
df_train_prep = apply_base_preprocessing(df_train)
df_train_prep

,id_ultimo_jefe,seniority,modalidad_trabajo,distancia_oficina,dias_baja_salud,genero,canal_reclutamiento,permanencia_promedio,salario,performance_score,psi_score,estado_civil,abandono_6meses
0,102074,1,Híbrida,1.760,1,Mujer,Ferias & Networking,1,140011,99,75,Soltero,0
1,102115,1,Híbrida,0.760,2,Hombre,Ferias & Networking,2,182774,99,81,Soltero,1
2,102060,1,Híbrida,4.950,3,Mujer,Referidos,11,682106,96,74,Viudo,0
3,102062,1,Híbrida,13.030,2,Hombre,Linkedin,2,270232,96,82,Soltero,1
4,102062,1,Híbrida,13.045,2,Hombre,Linkedin,2,266804,95,82,Soltero,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2147,102171,1,Presencial,1.965,8,Mujer,Portal Web,3,281159,0,65,Casado,1
2148,102172,1,Presencial,0.805,12,Mujer,Linkedin,13,460337,0,82,Casado,1
2149,102155,1,Presencial,1.625,1,Hombre,Portal Web,11,633879,0,66,Soltero,1
2150,102141,1,Presencial,5.665,2,Hombre,Portal Web,6,793977,0,80,Casado,1


## Entrenamiento y Validacion

In [18]:
# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# Validacion
from sklearn.model_selection import cross_val_score
# Preprocesamiento
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [48]:
# Seleccionar variables categoricas
cat_cols = [cname for cname in df_train_prep.columns if df_train_prep[cname].dtype == 'object']

# Transformador de columnas, esto es para hacer mas facil el preprocesamiento
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

models = {
    'tree': DecisionTreeClassifier(),
    'rdf': RandomForestClassifier(),
    'knn': KNeighborsClassifier(),
    'lgr': LogisticRegression()
}

In [20]:
# Separar columnas en 'Features' y 'Target'
X = df_train_prep.drop('abandono_6meses', axis=1)
y = df_train_prep.abandono_6meses

In [45]:
# Entrenar modelos y calcular f1-score usando cross-validation
# Se entre
for name, model in models.items():
    print(f'Training: {name}')
    pipe = Pipeline(steps=[
        ('prep', preprocessor),
        ('model', model)
    ])
    scores = cross_val_score(pipe, X, y, cv=5, scoring='f1')
    avg_score = round(np.mean(scores), 4)
    print(f'{name}: {avg_score}')
    print('---' * 3)

Training: tree
tree: 0.4715
---------
Training: rdf
rdf: 0.4849
---------
Training: knn
knn: 0.338
---------
Training: lgr
lgr: 0.4906
---------


Nos vamos con regresion logistica como modelo base para la primera entrega de las predictiones

## Prueba con Datos del Test

- Cargar datos de testeo
- Creacion del modelo con todos los datos de entrenamiento
- Predicciones en datos de testeo
- Creacion de prueba

In [37]:
# Cargar datos de testeo
df_test = pd.read_csv('data/test_data.csv', sep=';')
df_test.sample(5)

,id_colaborador,id_ultimo_jefe,seniority,modalidad_trabajo,distancia_oficina,dias_baja_salud,genero,canal_reclutamiento,permanencia_promedio,fecha_nacimiento,salario,performance_score,psi_score,fecha_incorporacion,estado_civil
1705,102808,102053.0,1,Presencial,2.385,1,Mujer,Referidos,4,18/07/1973,396418,85,79,18/08/2014,Casado
243,100124,102102.0,1,Híbrida,2.560,9,Mujer,Portal Web,11,17/11/1977,406373,81,83,31/07/2019,Casado
848,100692,102077.0,1,Presencial,2.870,1,Mujer,Portal Web,11,31/01/1980,373245,90,77,9/02/2017,Soltero
141,100737,102092.0,1,Híbrida,5.280,1,Mujer,Ferias & Networking,1,18/04/2000,121171,86,90,3/08/2018,Soltero
1239,103638,102111.0,1,Presencial,4.295,2,Hombre,Linkedin,4,6/11/1981,467515,91,87,4/04/2018,Casado


In [38]:
# Hacer una limpieza basica de los datos
df_test_prep = apply_base_preprocessing(df_test)

In [46]:
model = LogisticRegression()

# Creacion de pipeline (linea para hacer el proceso de preprocesamiento y entrenamiento)
# Ventajas: mas simple, evitamos errores en el preprocesamiento y escalable en el tiempo
pipe = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', model)
])

# Entrenamiento con todos los datos de entrenamiento
pipe.fit(X, y)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['modalidad_trabajo',
                                                   'genero',
                                                   'canal_reclutamiento',
                                                   'estado_civil'])])),
                ('model', LogisticRegression())])

In [39]:
# Hacer las predicciones con el modelo
preds = pipe.predict(df_test_prep)
preds[:10]

array([0, 1, 1, 0, 0, 0, 1, 0, 0, 1])

In [40]:
# Crear dataframe para enviar prueba
df_submission = pd.DataFrame({
    'ID': df_test.id_colaborador,
    'abandono_6meses': preds
})

df_submission.head()

,ID,abandono_6meses
0,100486,0
1,103752,1
2,103937,1
3,101744,0
4,101037,0


In [ ]:
# Guardar el dataframe en un archivo csv
# df_submission.to_csv('submissions/01_basemodel.csv', index=False)